<a href="https://colab.research.google.com/github/Dwarkamoye/Python-CS5590-490-0001/blob/master/LAB-2/txt_clsf_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer, one_hot
import pandas as pd
from keras_preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import numpy as np

Using TensorFlow backend.


In [0]:
df = pd.read_csv('SPAM - Data.csv',encoding='latin-1')
sentences = df['Message'].values
y = df['Category'].values


In [0]:
max_review_len= max([len(s.split()) for s in sentences])
#tokenizing data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)
#getting the vocabulary of data
sentences = tokenizer.texts_to_sequences(sentences)
padded_docs= pad_sequences(sentences,maxlen=max_review_len)

In [0]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.25, random_state=1000)
vocab_size= len(tokenizer.word_index)+1
num_classes = y_test.shape[0]

In [0]:
model = Sequential()
model.add(layers.Embedding(vocab_size, 50, input_length=max_review_len))
#model.add(layers.Flatten())
model.add(Conv1D(32, 3, activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=1))

model.add(Conv1D(128, 3, activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=1))


# flattening the matrix into vector form
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])

W0720 04:51:28.487202 139927249016704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 04:51:28.539994 139927249016704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 04:51:28.550397 139927249016704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0720 04:51:28.642632 139927249016704 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0720 04:51:28.665069 

In [0]:
model.fit(X_train,y_train, epochs=5, verbose=True, validation_data=(X_test,y_test), batch_size=64)


W0720 04:51:32.301690 139927249016704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 4179 samples, validate on 1393 samples
Epoch 1/5
4179/4179 [==============================] - 109s 26ms/step - loss: 2.4208 - acc: 0.8387 - val_loss: 1.9903 - val_acc: 0.8765
Epoch 2/5
4179/4179 [==============================] - 143s 34ms/step - loss: 2.2179 - acc: 0.8624 - val_loss: 1.9903 - val_acc: 0.8765
Epoch 3/5
4179/4179 [==============================] - 143s 34ms/step - loss: 2.2179 - acc: 0.8624 - val_loss: 1.9903 - val_acc: 0.8765
Epoch 4/5
4179/4179 [==============================] - 143s 34ms/step - loss: 2.2179 - acc: 0.8624 - val_loss: 1.9903 - val_acc: 0.8765
Epoch 5/5
4179/4179 [==============================] - 145s 35ms/step - loss: 2.2179 - acc: 0.8624 - val_loss: 1.9903 - val_acc: 0.8765


In [0]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("loss: %.2f%%" % (scores[0]*100))
model.save('./model' + '.h5')

Accuracy: 87.65%
loss: 199.03%
